#### 1. Setup

In [3]:
import openai
import os
import nest_asyncio

from dotenv import load_dotenv
from openai import OpenAI

# Load environment variables from .env file
load_dotenv()

# Allow for nested asyncio event loops
nest_asyncio.apply()

# Set up OpenAI API client
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

#### 2. Load Data

In [4]:
import os
from llama_index.core import SimpleDirectoryReader

# Define the base directory for documents relative to the root directory
base_dir = os.path.join(os.getcwd(), 'docs')

# List all files in the base directory
input_files = [os.path.join(base_dir, f) for f in os.listdir(base_dir) if os.path.isfile(os.path.join(base_dir, f))]

# Print the list of input files
print(f"Files to be loaded: {input_files}")

try:
    # Load documents from all files in the directory
    documents = SimpleDirectoryReader(input_files=input_files).load_data()
    
    # Check if documents are loaded
    if documents:
        print(f"Successfully loaded {len(documents)} document(s).")
        # Optionally, inspect the content of the first document
        print("First document content preview:")
        print(documents[0] if len(documents) > 0 else "No documents found.")
    else:
        print("No documents were loaded.")
except Exception as e:
    print(f"An error occurred while loading documents: {e}")


Files to be loaded: ['d:\\github-repo-tkhongsap\\editor-in-chief\\docs\\articles_part_00']
Successfully loaded 1 document(s).
First document content preview:
Doc ID: 5c0b711f-313e-45a7-b83d-1040edf72459
Text: id: 19050  attributes:      title: นโยบายเศรษฐกิจ ของพ่อนายกฯ
slug: politics-thaksin-shows-vision-economic-business      createdAt:
2024-08-23T11:11:39.120Z      updatedAt: 2024-08-23T11:12:58.030Z
publishedAt: 2024-08-23T11:11:41.837Z      locale: th      excerpt:
Deep SPACE – ไม่เชื่อ ก็ต้องเชื่อ
กับความเก๋าเกมทั้งด้านการเมืองและเศรษ...


#### 3. Define LLM and Embedding model

In [5]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [6]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model="gpt-4o")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-large")

#### 4. Define Summary Index and Vector Index over the Same Data

In [7]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

#### 5. Define Query Engines and Set Metadata

In [8]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()

In [9]:
from llama_index.core.tools import QueryEngineTool

summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions "
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context"
    ),
)

#### 6. Define Router Query Engine

In [12]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector


query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True
)

In [13]:

# Start a while loop to continuously get user input and query the engine
while True:
    user_input = input("Enter your query (or type 'exit' to quit): ")
    
    # Exit the loop if the user types 'exit'
    if user_input.lower() == 'exit':
        break
    
    # Perform the query
    response = query_engine.query(user_input)
    
    # Print the response in markdown format
    print(f"### Response:\n{str(response)}\n")


### Response:
The document is a compilation of various articles and reports covering a wide range of topics. These include political news, economic policies, natural disasters, business discussions, international collaborations, lifestyle and travel features, cultural content reviews, and social events. Specific subjects mentioned are economic visions and policies, natural disaster impacts, business challenges, political dynamics, cyber security, and environmental conservation. Additionally, it includes updates on government actions, legal issues, and significant political figures.

